In [ ]:
# Execute this cell to setup the python env in the Colab environment
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()
!rm -rf /usr/local/conda-meta/pinned # remove pins so we can use cuda 11.8
!mamba install torchmd-net=*=cuda118*
!mamba install openmm-torch=*=cuda118*
!mamba install -c conda-forge openmm-ml
!mamba install -c conda-forge openmmforcefields

In [ ]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from sys import stdout
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm
from openmmml import MLPotential
# this import registers the model with openmm-ml
from aceff_examples import torchmdnetpotential


#user supplied paths
sdfname = "ejm_31_ligand.sdf"
model_file = "../models/aceff_1.1.ckpt"


# Load the ligand with OpenFF toolkit
ligand = Molecule.from_file(sdfname)

# make an OpenFF Topology of the ligand
ligand_off_topology = offTopology.from_molecules(molecules=[ligand])

# get the total molecule charge
molecule_charge = int(sum( [ atom.formal_charge.magnitude for atom in ligand_off_topology.atoms]))

# convert it to an OpenMM Topology
ligand_omm_topology = ligand_off_topology.to_openmm()

# get the positions of the ligand
ligand_positions = offquantity_to_openmm(ligand.conformers[0])

# get the indicies
# this is a list of list so molecules can be batched if needed
group_indices=[list(range(ligand_off_topology.n_atoms))]

# create the ML potential with AceFF
potential = MLPotential('TorchMD-NET', model_file=model_file, group_indices=group_indices, molecule_charges=[molecule_charge], max_num_neighbors=40)

# create the OpenMM system
system = potential.createSystem(ligand_omm_topology)

integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 1.0*unit.femtoseconds)
simulation = app.Simulation(ligand_omm_topology, system, integrator, platform = mm.Platform.getPlatformByName('CUDA'))

# set the positions
simulation.context.setPositions(ligand_positions)

print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=1000)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

simulation.reporters.append(app.PDBReporter(f'traj.pdb', 1000))

simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, speed=True))

print("Running simulation...")
simulation.step(10000)





